In [44]:
import pandas as pd
import numpy as np
import random
import sys
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.decomposition import NMF
random.seed(4)

In [64]:
ratings = pd.read_csv('u.data',
                      sep = '\t',names= ['UserID', 'ItemID', 'rating', 'Timestamp' ])

test = ratings.sample(frac=.25)

In [65]:
ratings.loc[test.index, 'rating'] = None

In [66]:
test.head()

,UserID,ItemID,rating,Timestamp
42500,640,1016,3,886474538
46255,497,597,3,879310649
97132,401,188,1,891033267
49080,660,1074,1,891201399
35696,280,13,5,891700257


In [67]:
ratings.loc[11523]

UserID             387.0
ItemID             208.0
rating               3.0
Timestamp    886480484.0
Name: 11523, dtype: float64

In [68]:
ratings_matrix = ratings.pivot_table(index=['UserID'],columns=['ItemID'],values='rating', dropna=False)

avg_matrix = ratings.pivot_table(index=['UserID'],columns=['ItemID'],values='rating', dropna=False)

In [69]:
avg_matrix.fillna(avg_matrix.mean(), inplace=True)


In [70]:
avg_matrix.fillna(0, inplace=True)

In [71]:
preds = test.apply(lambda row: avg_matrix[row.ItemID][row.UserID], axis=1)
actual = test.apply(lambda row: row.rating, axis=1)

In [72]:
mean_squared_error(actual, preds)

1.0668650115420382

In [76]:
ks = [10, 20, 40, 80, 100]
scores = []
ratings_matrix.fillna(0, inplace=True)

In [79]:
for k in ks:
    model = NMF(n_components=k, random_state=None, max_iter=1500) 
    W = model.fit_transform(ratings_matrix)
    H = model.components_
    O = np.dot(W, H)
    k_preds = test.apply(lambda row: O[row.UserID-1][row.ItemID-1], axis=1)
    scores.append({k: mean_squared_error(actual, k_preds)})

C:\tools\Anaconda3\lib\site-packages\sklearn\decomposition\_nmf.py:1077: ConvergenceWarning: Maximum number of iterations 1500 reached. Increase it to improve convergence.
  " improve convergence." % max_iter, ConvergenceWarning)


In [80]:
scores

[{10: 7.266385662125529},
 {20: 7.217366096695849},
 {40: 7.78920202450557},
 {80: 8.810334696789788},
 {100: 9.179534284937002},
 {10: 7.2663858809718676},
 {20: 7.239380447977012},
 {40: 7.7352185250454975},
 {80: 8.854745256025048},
 {100: 9.223906728247526}]